# Создание дашборда по пользовательским событиям для агрегатора новостей


## Подготовка данных из БД

Прежде всего выгрузим данные из базы данных и познакомимся с ними. Далее, пордготовим csv-файл, с которым будем работать в Tableau.

Подключимся к базе данных через библиотеку Sqlalchemy.

In [1]:
# импортируем библиотеки
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

!pip install psycopg2-binary

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

Сформируем и выполним sql-запрос и сохраним результат выполнения в DataFrame.Sqlalchemy автоматически установит названия колоно такими же, как у таблицы в БД.

In [2]:
query = ''' SELECT *
            FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con = engine)

dash_visits.head(5)
 

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


## Знакомство с данными

Чуть подробнее познакомимся с имеющимися данными.

In [3]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [4]:
dash_visits.duplicated().sum()

0

In [5]:
dash_visits['record_id'].duplicated().sum()

0

In [6]:
dash_visits['item_topic'].unique()

array(['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье',
       'Знаменитости', 'Интересные факты', 'Искусство', 'История',
       'Красота', 'Культура', 'Наука', 'Общество', 'Отношения',
       'Подборки', 'Полезные советы', 'Психология', 'Путешествия',
       'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор'],
      dtype=object)

In [7]:
dash_visits['source_topic'].unique()


array(['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости',
       'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда',
       'Полезные советы', 'Политика', 'Психология', 'Путешествия',
       'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам',
       'Семейные отношения', 'Семья', 'Спорт', 'Строительство',
       'Технологии', 'Финансы'], dtype=object)

In [8]:
dash_visits['age_segment'].unique()


array(['18-25', '26-30', '31-35', '36-40', '41-45', '45+'], dtype=object)

In [9]:
dash_visits['dt'].min(), dash_visits['dt'].max()

(Timestamp('2019-09-24 18:28:00'), Timestamp('2019-09-24 19:00:00'))

In [10]:
dash_visits['visits'].min(), dash_visits['visits'].max(), dash_visits['visits'].median(), dash_visits['visits'].mean()

(1, 371, 3.0, 10.089673117580094)

Итак, мы имеем таблицу без явных дубликатов с информацией о более 30 тыс. событий взаимодействия пользователей 6 возрастных категорий с карточками в Яндекс.Дзене. Данные охватывают временной интервал около получаса 24 сентября 2019 года. Количество визитов в каждом наблюдении варьируется от 1 до 371, медианное значение - 3.

Таблица представлена в строко-ориентированном виде - как раз так, как нам нужно. Сформируем csv файл на основе которого будем строить дашборд в Tableau.

In [11]:
dash_visits.to_csv('dash_visits.csv', index=False)

## Формулировка ТЗ

<b>Мы имеем краткое ТЗ:</b>

- Бизнес-задача: <i>анализ взаимодействия пользователей с карточками Яндекс.Дзен</i>;
- Насколько часто предполагается пользоваться дашбордом: <i>не реже, чем раз в неделю</i>;
- Кто будет основным пользователем дашборда: <i>менеджеры по анализу контента</i>;
- Состав данных для дашборда:<i>
        1. История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
        2. Разбивка событий по темам источников;
        3. Таблица соответствия тем источников темам карточек;
</i>
- По каким параметрам данные должны группироваться:<i>
        1. Дата и время;
        2. Тема карточки;
        3. Тема источника;
        4. Возрастная группа;
</i>
- Характер данных:<i>
        1. История событий по темам карточек — абсолютные величины с разбивкой по минутам;
        2. Разбивка событий по темам источников — относительные величины (% событий);
        3. Соответствия тем источников темам карточек - абсолютные величины;
</i>
- Важность: <i>все графики имеют равную важность</i>;
- Источники данных для дашборда: <i>cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw)</i>;
- База данных, в которой будут храниться агрегированные данные: <i>дополнительные агрегированные таблицы в БД zen</i>;
- Частота обновления данных: <i>один раз в сутки, в полночь по UTC</i>;

Подготовим дашборд в Tableau в соответствии с макетом.


## Построение дашборда в Tableau

Ссылка на дашборд:

<https://public.tableau.com/app/profile/evgeniia.poddubnaia6108/viz/Book1_16643826037080/Dashboard1>